# 05 - Evaluation

En esta fase se evalúa el desempeño del modelo entrenado en la fase de *Modeling*  
usando métricas adecuadas y comparando los resultados con los objetivos definidos  
en la fase **01_business_understanding**.

---

## 1. Cargar librerías y datos


In [7]:
import os
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv("../data/03_primary/bmw_sales_clean.csv")
print("✅ Dataset cargado")
print("Columnas:", df.columns.tolist())
df.head()

✅ Dataset cargado
Columnas: ['Model', 'Year', 'Region', 'Color', 'Fuel_Type', 'Transmission', 'Engine_Size_L', 'Mileage_KM', 'Price_USD', 'Sales_Volume', 'Sales_Classification', 'car_age']


,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification,car_age
0,5 Series,2016,Asia,Red,Petrol,Manual,3.5,151748,98740,8300,High,9
1,i8,2013,North America,Red,Hybrid,Automatic,1.6,121671,79219,3428,Low,12
2,5 Series,2022,North America,Blue,Petrol,Automatic,4.5,10991,113265,6994,Low,3
3,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low,1
4,7 Series,2020,South America,Black,Diesel,Manual,2.1,122131,49898,3080,Low,5


In [10]:
X = df.drop(columns=["Price_USD"])
y = df["Price_USD"]

print("X shape:", X.shape, "| y shape:", y.shape)

X shape: (50000, 11) | y shape: (50000,)


In [11]:
# One-hot encoding (mismas reglas que en 04)
X = pd.get_dummies(X, drop_first=True)

print("✅ One-hot encoding aplicado")
print("Total de features tras OHE:", X.shape[1])
X.head()

✅ One-hot encoding aplicado
Total de features tras OHE: 30


,Year,Engine_Size_L,Mileage_KM,Sales_Volume,car_age,Model_5 Series,Model_7 Series,Model_M3,Model_M5,Model_X1,...,Color_Blue,Color_Grey,Color_Red,Color_Silver,Color_White,Fuel_Type_Electric,Fuel_Type_Hybrid,Fuel_Type_Petrol,Transmission_Manual,Sales_Classification_Low
0,2016,3.5,151748,8300,9,True,False,False,False,False,...,False,False,True,False,False,False,False,True,True,False
1,2013,1.6,121671,3428,12,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,True
2,2022,4.5,10991,6994,3,True,False,False,False,False,...,True,False,False,False,False,False,False,True,False,True
3,2024,1.7,27255,4047,1,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,True
4,2020,2.1,122131,3080,5,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,True


In [13]:
# Paso 6 - Preparar X_test con las mismas columnas que en el entrenamiento
import joblib
import os
import pandas as pd

# Cargar dataset limpio
df = pd.read_csv("../data/03_primary/bmw_sales_clean.csv")

# Separar features y target
X = df.drop(columns=["Price_USD"])
y = df["Price_USD"]

# Dividir en train/test igual que en 04
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# OneHotEncoding para variables categóricas
X_test_encoded = pd.get_dummies(X_test)
X_train_encoded = pd.get_dummies(X_train)

# Asegurar que X_test tenga las mismas columnas que X_train
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

print("✅ X_test preparado con las mismas columnas que en el entrenamiento")
print("Columnas finales:", X_test_encoded.shape[1])


✅ X_test preparado con las mismas columnas que en el entrenamiento
Columnas finales: 36


In [15]:
# Si el modelo tiene el atributo feature_names_in_, lo usamos para reordenar/crear faltantes
if hasattr(model, "feature_names_in_"):
    X_aligned = X.reindex(columns=model.feature_names_in_, fill_value=0)
else:
    # Fallback: si no existe el atributo (versiones antiguas), usamos el orden actual
    X_aligned = X.copy()

print("✅ Columnas alineadas con el modelo")
print("Coinciden las columnas?", list(X_aligned.columns) == list(getattr(model, "feature_names_in_", X_aligned.columns)))

✅ Columnas alineadas con el modelo
Coinciden las columnas? True


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X_aligned, y, test_size=0.2, random_state=42
)

print("Split listo ->",
      "X_train:", X_train.shape,
      "| X_test:", X_test.shape)

Split listo -> X_train: (40000, 30) | X_test: (10000, 30)


In [17]:
# Predicciones
y_pred = model.predict(X_test)

# Métricas
mse = mean_squared_error(y_test, y_pred)
r2  = r2_score(y_test, y_pred)

print("📊 Métricas de evaluación")
print(f"➡️ MSE: {mse:.2f}")
print(f"➡️ R² : {r2:.3f}")

📊 Métricas de evaluación
➡️ MSE: 677415141.31
➡️ R² : -0.001


In [ ]:
# Predicho vs Real
plt.figure(figsize=(7,6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.xlabel("Valor real (Price_USD)")
plt.ylabel("Predicción")
plt.title("Predicción vs Real - Regresión Lineal")
plt.grid(True, alpha=0.2)
plt.show()

# Distribución de residuos
residuos = y_test - y_pred
plt.figure(figsize=(7,6))
sns.histplot(residuos, bins=30, kde=True)
plt.xlabel("Error (residuo)")
plt.title("Distribución de errores")
plt.grid(True, alpha=0.2)
plt.show()

print("✅ Evaluación completada")

In [ ]:
### Conclusiones de la evaluación
- El modelo de Regresión Lineal logra explicar un **R² = ...**, lo que significa que captura X% de la varianza de los precios.
- El error medio cuadrático (MSE) indica que el modelo tiene un error promedio de ... USD².
- Los gráficos muestran que:
  - La mayoría de los puntos siguen la diagonal → buen ajuste general.
  - Los residuos se distribuyen de manera relativamente simétrica alrededor de 0 → sin sesgo evidente.
- El modelo puede mejorarse probando:
  - Algoritmos más complejos (Random Forest, XGBoost).
  - Mejor selección de features y tratamiento de outliers.
